In [1]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from pprint import pprint
from langchain_core.pydantic_v1 import BaseModel, Field

/home/wpnx/CODES/envs_global/LLM_BASE/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
### llm Gemma2 Ollama
llm =  ChatOllama(model="cow/gemma2_tools:2b")

In [3]:
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [19]:
### Parsing WebURL using langchain Website loader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Munich")

In [20]:
# lading website
raw_docs = loader.load()

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


In [21]:
# embedding model
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [22]:
# Text Splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=150, length_function=len,separators=["\n\n","\n"])

In [9]:
DOCS = splitter.split_documents(raw_docs)

In [10]:
len(DOCS)

295

In [11]:
DB = FAISS.from_documents(DOCS, embed_model)

In [12]:
DB.similarity_search("Do people use bicycles in Munich?",k=3)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Munich', 'title': 'Munich - Wikipedia', 'language': 'en'}, page_content="Cycling[edit]\nMain article: Cycling in Munich\nMap of Munich's cycling network\nCycling has a strong presence in the city and is recognized as a good alternative. The growing number of bicycle lanes are widely used throughout the year. Cycle paths can be found alongside the majority of sidewalks and streets, although the newer or renovated ones are much easier to tell apart from pavements than older ones. A modern bike hire system is available within the area bounded by the Mittlerer Ring."),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Munich', 'title': 'Munich - Wikipedia', 'language': 'en'}, page_content='Cultural history trails and bicycle routes[edit]\nSince 2001, historically interesting places in Munich can be explored via the List of cultural history trails in Munich (KulturGeschichtsPfade). Sign-posted cycle routes are the Outer

In [23]:
### Prompt
template = """
You are a helful bot. Your task is to find the answer from the given context.
If answer not in the context just say I dont know. 
Dont assume answers if not int context.

Context : {context}
Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [14]:
def format_docs(question):
    docs = [i.page_content for i in DB.similarity_search(question)]
    docs = "\n\n".join(docs)
    return docs

In [24]:
### DB and LLM for Question Answer
from langchain_core.runnables import RunnablePassthrough
chain = ({"context": format_docs, "question": RunnablePassthrough() } | prompt | llm |  StrOutputParser())

In [16]:
chain.invoke("Do people use bicycles in Munich?")

'Yes, people use bicycles in Munich.  The text states that cycling has a "strong presence" in the city and is recognized as a good alternative. \n'

In [18]:
chain.invoke("Give me some information about munich city ?")

"Here is some information about Munich, as it is described in the text:\r\n* **Location:** Straddling the banks of the river Isar north of the Alps.\r\n* **Demographics:**  It is the second-largest city in the Bavarian dialect area, after Vienna, and is the most densely populated municipality in Germany with 4,500 people per km2. \r\n* **Government:** It's the seat of the Bavarian administrative region of Upper Bavaria. \r\n* **Known for:**  Munich is home to various iconic landmarks such as Marienplatz, Viktualienmarkt and Frauenkirche (St Mary's). \n* **Other points of interest:** The text lists a number of museums, theatres, cultural institutions, and monuments that are located in Munich.\n\n\n\nLet me know what other information you would like! \r\n"